#### Loaded: Oracle Functions:
for getting:
- Percentage of remaining storage in Oracle
- Percentage of erroneous Oracle jobs
- Unique Active Users
- Availability of Oracle

In [0]:
import json 
import pandas as pd
from datetime import datetime

In [0]:
def generate_dataframe(config,query):
    configured_url = config['jdbc_url'].format(config['host'], config['port'])

    connectionProperties = {
        "user": config['user'],
        "password": config['password'],
        "driver": config['driver']
    }

    user_system_created_df=spark.read.jdbc(url=configured_url
                                           ,table=query
                                           ,properties=connectionProperties
                                           ,numPartitions=1)
    
    return user_system_created_df

In [0]:
#Percentage of remaining storage in Oracle
#
#Note: Cannot be run historically - simply queries Oracle as it is at runtime.
#Therefore, if we want this data on **DAY** granularity, then this code will need to be run every day.
#Alternatively, if we settle for *week* granularity, then this can be run once per week
#

def oracle_get_remaining_storage(oracle_config):
    remaining_storage_query = f'''(
        SELECT
        SUM(free_mb) AS "oracle_free_mb"
        ,SUM(total_mb) AS "oracle_total_mb"
        --,SUM(free_mb)/SUM(total_mb) AS "oracle_pct_mb_free"
        FROM v$asm_diskgroup
    )'''
    
    return generate_dataframe(oracle_config, remaining_storage_query)

In [0]:
#Percentage of erroneous Oracle jobs
def oracle_get_erroneous_jobs(oracle_config,first_date_str,last_date_str):
    pct_erroneous_jobs_query = f'''
        (SELECT count(case when status = 'FAILED' then 1 end) AS "oracle_failed_jobs"
        ,count(status) AS "oracle_total_jobs"
        ,TO_CHAR(CAST(log_date as date),'YYYY-MM-DD') as "date"
        FROM DBA_SCHEDULER_JOB_RUN_DETAILS
        WHERE TRUNC(log_date) BETWEEN to_date('{first_date_str} 00:00:00', 'YYYY-MM-DD hh24:mi:ss')
        AND to_date('{last_date_str} 23:59:59', 'YYYY-MM-DD hh24:mi:ss')
        GROUP BY TO_CHAR(CAST(log_date as date),'YYYY-MM-DD')
        ORDER BY TO_CHAR(CAST(log_date as date),'YYYY-MM-DD')
    )'''
    return generate_dataframe(oracle_config, pct_erroneous_jobs_query)

In [0]:
# Unique Active Users over given time span
# Un-comment SQL lines below to aggregate by day

def oracle_get_active_users(oracle_config,first_date_str,last_date_str):
    active_users_query = f'''(SELECT count(distinct s.username) as "oracle_active_users"
        --,TO_CHAR(CAST(s.TIMESTAMP as date),'YYYY-MM-DD') as "date"
        FROM DBA_AUDIT_SESSION s
        JOIN DBA_USERS u ON s.USERNAME = u.USERNAME
        WHERE s.TIMESTAMP BETWEEN to_date('{first_date_str}', 'YYYY-MM-DD')
        AND to_date('{last_date_str} 23:59:59', 'YYYY-MM-DD hh24:mi:ss')
        AND u.PROFILE = 'ZALANDO_DEVELOPER'
        --GROUP BY TO_CHAR(CAST(s.TIMESTAMP as date),'YYYY-MM-DD')
        --ORDER BY TO_CHAR(CAST(s.TIMESTAMP as date),'YYYY-MM-DD')
    )'''
    return generate_dataframe(oracle_config, active_users_query)

In [0]:
# Oracle Availability

def get_oracle_availability(oracle_config,first_date_str,last_date_str,minimum_downtime_in_minutes = 1):
    availability_query = f'''(
    WITH timestamp_gaps AS (
        SELECT ACTUAL_START_DATE,
        LAG(ACTUAL_START_DATE) OVER (ORDER BY ACTUAL_START_DATE ASC) AS "PREV_START_DATE"
        FROM DBA_SCHEDULER_JOB_RUN_DETAILS
        WHERE TRUNC(ACTUAL_START_DATE) BETWEEN to_date('{first_date_str}', 'YYYY-MM-DD')
        AND to_date('{last_date_str}', 'YYYY-MM-DD')
    )
    SELECT
    SUM(
        CASE
            WHEN (EXTRACT(second FROM (ACTUAL_START_DATE - PREV_START_DATE))/60 > {minimum_downtime_in_minutes})
            THEN EXTRACT(second FROM (ACTUAL_START_DATE - PREV_START_DATE))/60
            ELSE 0
        END
    ) AS "oracle_downtime_in_mins"
    ,TO_CHAR(TRUNC(ACTUAL_START_DATE),'YYYY-MM-DD') AS "date"
    FROM timestamp_gaps
    WHERE ACTUAL_START_DATE IS NOT NULL
    AND PREV_START_DATE IS NOT NULL
    GROUP BY TRUNC(ACTUAL_START_DATE)
    ORDER BY TRUNC(ACTUAL_START_DATE)
    )'''
    
    return generate_dataframe(oracle_config, availability_query)
